In [345]:
import time

import tensorflow as tf
import numpy as np

from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib

from keras.datasets import cifar10, mnist

# Write TFRecord

In [286]:
# Modified code from TensorFlow Repo
# Reference: tensorflow/examples/how_tos/reading_data/convert_to_records.py
def _int64_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to(data_set, name):
    """Converts a dataset to tfrecords."""
    images = data_set[0]
    labels = data_set[1]
    num_examples = data_set[2]

    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d.' %
                     (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    if len(images.shape) == 4:
        depth = images.shape[3]
    else:
        depth = 1

    filename =  name + '.tfrecords'
    print('Writing', filename, end=' ')
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()
    print('done.')


In [287]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [359]:
y_test.shape

(10000, 1)

In [384]:
convert_to([x_train, y_train, len(x_train)], 'train')
convert_to([x_test, y_test, len(x_test)], 'test')

Writing train.tfrecords done.
Writing test.tfrecords done.


# Read TFRecord

In [332]:
# Modified code from TensorFlow Repo
# Reference: tensorflow/examples/how_tos/reading_data/fully_connected_reader.py

def read_and_decode(filename_queue, shape):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={'image_raw': tf.FixedLenFeature([], tf.string),
                                                 'label': tf.FixedLenFeature([], tf.float32),})

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.reshape(image, shape)
    #image.set_shape([image_pixels])

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)
    #label = tf.reshape(image, [1])
    return image, label


def inputs(train, batch_size, num_epochs, filename, shape):
    if not num_epochs: num_epochs = None

    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
        [filename], num_epochs=num_epochs)

    # Even when reading in multiple threads, share the filename
    # queue.
    image, label = read_and_decode(filename_queue, shape)

    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in two threads to avoid being a bottleneck.
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)
    #sparse_labels = tf.reshape(sparse_labels, [-1, 10])
    return images, sparse_labels

In [333]:
images, labels = inputs(train=True, batch_size=128,  num_epochs=5, 
                        filename='./train.tfrecords', shape=[32, 32, 3])

In [292]:
images

<tf.Tensor 'shuffle_batch_1:0' shape=(128, 32, 32, 3) dtype=float32>

In [293]:
labels

<tf.Tensor 'shuffle_batch_1:1' shape=(128, 10) dtype=float32>

# Build the model

In [346]:
def alex_net(inputs, n_classes, prob=1., verbose=False):
    input_layer = tf.reshape(inputs, [-1, 32, 32, 3])
    print(input_layer) if verbose else 0
    net = layers.conv2d(input_layer, 64, [11, 11], 3, padding='SAME', scope='conv1', activation_fn=tf.nn.relu, \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool1')
    print(net) if verbose else 0
    net = layers.conv2d(net, 192, [3, 3], scope='conv2', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 1, scope='pool2')
    print(net) if verbose else 0
    net = layers.conv2d(net, 384, [3, 3], scope='conv3', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.conv2d(net, 384, [3, 3], scope='conv4', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.conv2d(net, 256, [3, 3], scope='conv5', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool5')
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=4096, scope='fc1', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.dropout(net, prob)
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=4096, scope='fc2', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.dropout(net, prob)
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=n_classes, scope='fc3', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.softmax(net)
    print(net) if verbose else 0
    return tf.reshape(net, [-1, 10])

# Train

In [400]:
def train(learning_rate=0.01, training_epochs=1, batch_size=128, \
          display_step=1, logs_path='./logs', keep_probability= 0.5, num_samples=50000, training_time=120):

    tf.reset_default_graph()
    
    x, y = inputs(train=True, batch_size=batch_size, num_epochs=training_epochs*2, \
                                filename='./train.tfrecords', shape=[32, 32, 3])
    x_test, y_test = inputs(train=False, batch_size=batch_size, num_epochs=training_epochs*2, \
                                filename='./test.tfrecords', shape=[32, 32, 3])

    pred = alex_net(x, 10, keep_probability)
    pred_test = alex_net(x_test, 10, keep_probability)
    
    #Cross entropy loss
    loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred+1e-9), reduction_indices=1))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    accuracy_train = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), tf.float32))
    accuracy_test = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred_test, 1), tf.argmax(y_test, 1)), tf.float32))
    
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    tf.summary.scalar("Loss", loss)
    tf.summary.scalar("Accuracy_Train", accuracy_train)
    tf.summary.scalar("Accuracy_Test", accuracy_test)
    
    merged_summary_op = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print('Num of parameters:', total_parameters)
    
    with tf.Session() as session:
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        session.run(init)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=session, coord=coord)
        
        step = 0
        start_time = time.time()

        total_iterations = int(num_samples / batch_size * training_epochs)
        print('Num of steps:', total_iterations)
        for step in range(total_iterations):
            _, loss_value, summary = session.run([optimizer, loss, merged_summary_op])
            duration = time.time() - start_time         
            if step % display_step == 0:
                print('Step %d: loss = %.2f - accuracy train = %.2f - accuracy test =  %.2f (%.3f sec)' % 
                          (step, loss_value, accuracy_train.eval(), accuracy_test.eval(), duration))
            if duration/60 > training_time: 
                print('Training time exceeded. Stopping now...')
                break
            summary_writer.add_summary(summary, step)
            saver.save(sess, './models/model_' + str(learning_rate) + '_' + str(batch_size))
        coord.request_stop()
        coord.join(threads)
        session.close()

In [401]:
train(batch_size=128, display_step=1, training_epochs=100, num_samples=50000)

Num of parameters: 20885450
Num of steps: 39062
Step 0: loss = 19.27 - accuracy train = 0.10 - accuracy test =  0.13 (2.176 sec)
Step 1: loss = 18.29 - accuracy train = 0.13 - accuracy test =  0.11 (4.541 sec)
Step 2: loss = 17.81 - accuracy train = 0.11 - accuracy test =  0.07 (7.127 sec)
Step 3: loss = 19.27 - accuracy train = 0.11 - accuracy test =  0.12 (9.671 sec)
Step 4: loss = 18.29 - accuracy train = 0.11 - accuracy test =  0.07 (12.183 sec)
Step 5: loss = 18.94 - accuracy train = 0.13 - accuracy test =  0.09 (14.775 sec)
Step 6: loss = 18.46 - accuracy train = 0.12 - accuracy test =  0.08 (17.714 sec)
Step 7: loss = 18.78 - accuracy train = 0.14 - accuracy test =  0.02 (21.164 sec)
Step 8: loss = 18.22 - accuracy train = 0.05 - accuracy test =  0.10 (24.610 sec)
Step 9: loss = 19.27 - accuracy train = 0.09 - accuracy test =  0.09 (27.394 sec)
Step 10: loss = 19.27 - accuracy train = 0.15 - accuracy test =  0.10 (30.096 sec)
Step 11: loss = 18.62 - accuracy train = 0.12 - accur

![](./images/loss.png)

![](./images/train.png)

![](./images/test.png)